In [3]:
# Import libraries 
import pandas as pd
import numpy as np
##import warnings 
#warnings.filterwarnings('ignore')

In [4]:
# Import dataset 
df = pd.read_csv(r'E:\MLproject\powergauge\notebook\data\Homestead Electricity Consumption.csv')

df.head()

,Maximum Temperature,Minimum Temperature,Dew Point,Heat Index,Wind Chill,Wind Gust Kmph,Cloud Cover,Humidity,Precipitation MM,Pressure,Temperature,Visibility,Wind Direction Degree,Wind speed Kmph,Consumption
0,33,27,23,36,31,24,9,62,0.0,1018,31,10,94,19,103
1,33,27,23,37,32,24,7,60,0.0,1018,32,10,96,20,99
2,33,27,23,38,33,23,5,57,0.0,1019,33,10,98,20,74
3,33,27,23,37,32,22,6,57,0.0,1019,32,10,96,19,82
4,33,27,23,37,32,21,7,58,0.0,1019,32,10,95,18,73


In [5]:
# Data Summary 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22201 entries, 0 to 22200
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Maximum Temperature    22201 non-null  int64  
 1   Minimum Temperature    22201 non-null  int64  
 2   Dew Point              22201 non-null  int64  
 3   Heat Index             22201 non-null  int64  
 4   Wind Chill             22201 non-null  int64  
 5   Wind Gust Kmph         22201 non-null  int64  
 6   Cloud Cover            22201 non-null  int64  
 7   Humidity               22201 non-null  int64  
 8   Precipitation MM       22201 non-null  float64
 9   Pressure               22201 non-null  int64  
 10  Temperature            22201 non-null  int64  
 11  Visibility             22201 non-null  int64  
 12  Wind Direction Degree  22201 non-null  int64  
 13  Wind speed Kmph        22201 non-null  int64  
 14  Consumption            22201 non-null  int64  
dtypes:

In [6]:
# # Drop columns 
# df = df.drop(columns={'Unnamed: 0', 'Date'})
# df.head()

In [7]:
# # Rename columns 
# df = df.rename(columns={'Homestead_maxtempC': 'Max Temp °C', 'Homestead_mintempC':'Min temp °C', 
#                         'Homestead_DewPointC':'Dewpoint °C', 'Homestead_FeelsLikeC':'Feels Like °C',
#                         'Homestead_HeatIndexC':'Heat Index °C', 'Homestead_WindChillC':'Wind Chill °C', 
#                         'Homestead_WindGustKmph':'Wind Gust Kmph', 'Homestead_cloudcover':'Cloud Cover',
#                         'Homestead_humidity':'Humidity', 'Homestead_precipMM':'Precipitation MM',
#                         'Homestead_pressure':'Pressure', 'Homestead_tempC':'Temp °C', 'Homestead_visibility':'Visibility',
#                         'Homestead_winddirDegree':'Wind Direction Degree', 'Homestead_windspeedKmph':'Wind speed Kmph'})

# df.head()

In [8]:
print(df.columns)

Index(['Maximum Temperature ', 'Minimum Temperature', 'Dew Point ',
       'Heat Index', 'Wind Chill', 'Wind Gust Kmph', 'Cloud Cover', 'Humidity',
       'Precipitation MM', 'Pressure', 'Temperature ', 'Visibility',
       'Wind Direction Degree', 'Wind speed Kmph', 'Consumption'],
      dtype='object')


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Evaluation 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
# Define the features and target variable 

# drop null values
df = df.dropna()

# Features 
X = df.drop(columns={'Consumption'})

# Target variable 
y = df['Consumption']

In [16]:
# Training set and testing split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
# Initialize the standard scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data
X_test_scaled = scaler.transform(X_test)

In [25]:
from sklearn.model_selection import StratifiedKFold


# Define the XGBoost model
xgb_model = XGBRegressor()

# Define the number of folds for cross-validation
n_folds = 5

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics
mae_scores = []
r2_scores = []
mape_scores = []
mse_scores = []
rmse_scores = []

# Perform stratified k-fold cross-validation
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the standard scaler
    scaler = StandardScaler()

    # Fit and transform the training data
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform the testing data
    X_test_scaled = scaler.transform(X_test)


    # Fit the model on the training data
    xgb_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = xgb_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Store the evaluation metrics
    mae_scores.append(mae)
    r2_scores.append(r2)
    mape_scores.append(mape)
    mse_scores.append(mse)
    rmse_scores.append(rmse)

# Calculate the average evaluation metrics
mae_avg = np.mean(mae_scores)
r2_avg = np.mean(r2_scores)
mape_avg = np.mean(mape_scores)
mse_avg = np.mean(mse_scores)
rmse_avg = np.mean(rmse_scores)

# Print the average evaluation metrics
print("MAE:", mae_avg)
print("R2:", r2_avg)
print("MAPE:", mape_avg)
print("MSE:", mse_avg)
print("RMSE:", rmse_avg)

MAE: 8.01223442059759
R2: 0.6854020439809962
MAPE: 13.206206084213225
MSE: 103.43808278316196
RMSE: 10.169962287078185


In [27]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import optuna
import numpy as np

# Define the objective function for Optuna
def objective(trial):
    # Define the number of folds for cross-validation
    n_folds = 5

    # Create a StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Initialize lists to store the evaluation metrics
    mae_scores = []

    # Define hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 200, step=10),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100),
        'gamma': trial.suggest_float('gamma', 1e-5, 10)
    }

    # Initialize the model with the suggested hyperparameters
    xgb_model = XGBRegressor(**params)

    # Perform stratified k-fold cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize the standard scaler
        scaler = StandardScaler()

        # Fit and transform the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Fit the model on the training data
        xgb_model.fit(X_train_scaled, y_train)

        # Make predictions on the testing data
        y_pred = xgb_model.predict(X_test_scaled)

        # Evaluate the model on the testing data using MAE
        mae = mean_absolute_error(y_test, y_pred)

        # Store the evaluation metric (MAE)
        mae_scores.append(mae)

    # Calculate the average MAE across all folds
    avg_mae = np.mean(mae_scores)

    # Return the negative of the average MAE as Optuna aims to minimize the objective
    return -avg_mae




In [28]:
# Create the Optuna study object and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and the corresponding MAE
best_params = study.best_params
best_mae = -study.best_value
print("Best Hyperparameters:", best_params)
print("Best MAE:", best_mae)

[I 2024-03-12 21:35:27,150] A new study created in memory with name: no-name-ea897dde-9f73-472d-b572-8ad8f58f6257
[I 2024-03-12 21:35:31,056] Trial 0 finished with value: -12.005803028753208 and parameters: {'n_estimators': 30, 'max_depth': 3, 'learning_rate': 0.03849744842304364, 'subsample': 0.7312632733363187, 'colsample_bytree': 0.8124133845335217, 'reg_alpha': 5.902643472268857, 'reg_lambda': 39.303633076184994, 'gamma': 3.4057652631666397}. Best is trial 0 with value: -12.005803028753208.
[I 2024-03-12 21:35:37,639] Trial 1 finished with value: -8.551260157551596 and parameters: {'n_estimators': 190, 'max_depth': 9, 'learning_rate': 0.045333161737342954, 'subsample': 0.8626121341617912, 'colsample_bytree': 0.8929556673468231, 'reg_alpha': 78.40642254595653, 'reg_lambda': 63.732208147522314, 'gamma': 3.523981540512617}. Best is trial 0 with value: -12.005803028753208.
[I 2024-03-12 21:35:45,022] Trial 2 finished with value: -7.558037013947247 and parameters: {'n_estimators': 200, 

Best Hyperparameters: {'n_estimators': 20, 'max_depth': 4, 'learning_rate': 0.010100010973689563, 'subsample': 0.5846588292676513, 'colsample_bytree': 0.6584575863282478, 'reg_alpha': 44.36226230599742, 'reg_lambda': 39.94702375399262, 'gamma': 0.04548674273836721}
Best MAE: 14.089200642384123


In [19]:
def model(X_train_scaled, y_train, X_test_scaled, y_test):
    # Initialize the XGBRegressor model
    rf_model = RandomForestRegressor()

    # Fit the model on the entire training data
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = rf_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)

    # Return the evaluation metrics
    return mae_test, r2_test, mape_test, mse_test, rmse_test

# Example usage:
# Assuming you have defined X_train_scaled, y_train, X_test_scaled, and y_test
mae_test, r2_test, mape_test, mse_test, rmse_test = model(X_train_scaled, y_train, X_test_scaled, y_test)

print("MAE:", mae_test)
print("R2:", r2_test)
print("MAPE:", mape_test)
print("MSE:", mse_test)
print("RMSE:", rmse_test)

MAE: 7.347431093496667
R2: 0.7271726711754497
MAPE: 12.150934498611557
MSE: 90.30438690326068
RMSE: 9.502862037473799


In [21]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [22]:
def model(X_train_scaled, y_train, X_test_scaled, y_test):
    # Initialize the XGBRegressor model
    et_model = ExtraTreesRegressor()

    # Fit the model on the entire training data
    et_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = et_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)

    # Return the evaluation metrics
    return mae_test, r2_test, mape_test, mse_test, rmse_test

# Example usage:
# Assuming you have defined X_train_scaled, y_train, X_test_scaled, and y_test
mae_test, r2_test, mape_test, mse_test, rmse_test = model(X_train_scaled, y_train, X_test_scaled, y_test)

print("MAE:", mae_test)
print("R2:", r2_test)
print("MAPE:", mape_test)
print("MSE:", mse_test)
print("RMSE:", rmse_test)

MAE: 6.799192938209331
R2: 0.7560882659409939
MAPE: 11.235022412076367
MSE: 80.73347966132229
RMSE: 8.985181114553134


In [24]:
from sklearn.model_selection import StratifiedKFold

# Initialize the XGBRegressor model
et_model = ExtraTreesRegressor()

# Define the number of folds for cross-validation
n_folds = 5

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics
mae_scores = []
r2_scores = []
mape_scores = []
mse_scores = []
rmse_scores = []

# Perform stratified k-fold cross-validation
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    # Initialize the standard scaler
    scaler = StandardScaler()

    # Fit and transform the training data
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform the testing data
    X_test_scaled = scaler.transform(X_test)


    # Fit the model on the training data
    et_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = et_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Store the evaluation metrics
    mae_scores.append(mae)
    r2_scores.append(r2)
    mape_scores.append(mape)
    mse_scores.append(mse)
    rmse_scores.append(rmse)

# Calculate the average evaluation metrics
mae_avg = np.mean(mae_scores)
r2_avg = np.mean(r2_scores)
mape_avg = np.mean(mape_scores)
mse_avg = np.mean(mse_scores)
rmse_avg = np.mean(rmse_scores)

# Print the average evaluation metrics
print("MAE:", mae_avg)
print("R2:", r2_avg)
print("MAPE:", mape_avg)
print("MSE:", mse_avg)
print("RMSE:", rmse_avg)

MAE: 6.746451810879783
R2: 0.75649427917282
MAPE: 11.141870865995454
MSE: 80.06279018643258
RMSE: 8.947409056935156


In [29]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import optuna
import numpy as np

# Define the objective function for Optuna
def objective(trial):
    # Define the number of folds for cross-validation
    n_folds = 5

    # Create a StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Initialize lists to store the evaluation metrics
    mae_scores = []

    # Define hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 200, step=10),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }

    # Initialize the model with the suggested hyperparameters
    et_model = ExtraTreesRegressor(**params)

    # Perform stratified k-fold cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize the standard scaler
        scaler = StandardScaler()

        # Fit and transform the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Fit the model on the training data
        et_model.fit(X_train_scaled, y_train)

        # Make predictions on the testing data
        y_pred = et_model.predict(X_test_scaled)

        # Evaluate the model on the testing data using MAE
        mae = mean_absolute_error(y_test, y_pred)

        # Store the evaluation metric (MAE)
        mae_scores.append(mae)

    # Calculate the average MAE across all folds
    avg_mae = np.mean(mae_scores)

    # Return the negative of the average MAE as Optuna aims to minimize the objective
    return -avg_mae




In [30]:
# Create the Optuna study object and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Print the best hyperparameters and the corresponding MAE
best_params = study.best_params
best_mae = -study.best_value
print("Best Hyperparameters:", best_params)
print("Best MAE:", best_mae)

[I 2024-03-12 21:40:50,726] A new study created in memory with name: no-name-8bc0ca1a-05ce-475e-aafe-9607dab32442
[I 2024-03-12 21:40:54,367] Trial 0 finished with value: -12.12727358086765 and parameters: {'n_estimators': 120, 'max_depth': 6, 'min_samples_split': 11, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: -12.12727358086765.
[I 2024-03-12 21:41:00,070] Trial 1 finished with value: -11.634264091427983 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: -12.12727358086765.
[I 2024-03-12 21:41:06,146] Trial 2 finished with value: -11.446136927756973 and parameters: {'n_estimators': 190, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: -12.12727358086765.
[I 2024-03-12 21:41:06,970] Trial 3 finished with value: -12.384675807127035 and parameters: {'n_estimators': 40, 'max_depth': 5, 'm

Best Hyperparameters: {'n_estimators': 20, 'max_depth': 5, 'min_samples_split': 20, 'min_samples_leaf': 8, 'max_features': 'sqrt'}
Best MAE: 12.444356476378672
